In [6]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text
from sqlalchemy.sql import func, distinct, exists
engine = create_engine('sqlite+pysqlite:////Users/vladislavvorosilov/Desktop/Python_SQL/Python_SQL_PI_2021_22_s2/Data/Students.db')
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine) 
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")
conn=engine.connect()


### 1. Напишите запрос, который выполняет вывод фамилий студентов, сдававших экзамены, вместе с идентификаторами каждого сданного ими предмета обучения.

In [6]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.subj_id)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id)


result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

47


[('Вишнякова', 'Анжела', 10),
 ('Казакова', 'Мария', 73),
 ('Жук', 'Оксана', 73),
 ('Жуков', 'Пров', 56),
 ('Кондрат', 'Синицин', 10),
 ('Лазарев', 'Константин', 56),
 ('Афанасьева', 'Анжелика', 12),
 ('Федоров', 'Гостомысл', 12),
 ('Блохина', 'Ираида', 22),
 ('Суханов', 'Константин', 22)]

### 2. Напишите запрос, который выполняет выборку фамилий всех студентов, с указанием для студентов, сдававших экзамены, идентификаторов сданных ими предметов обучения.

In [7]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.subj_id)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id, isouter=True)


result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

100


[('Иванов', 'Иван', 43),
 ('Петров', 'Петр', None),
 ('Сидоров', 'Вадим', 22),
 ('Кузнецов', 'Борис', 12),
 ('Зайцева', 'Ольга', 10),
 ('Зайцева', 'Ольга', 10),
 ('Ордуханян', 'Рафаэль', 10),
 ('Котов', 'Павел', 10),
 ('Белкин', 'Вадим', 10),
 ('Белкин', 'Вадим', 10)]

### 3. Напишите запрос, который выполняет вывод фамилий студентов, сдававших экзамены, вместе с наименованиями каждого сданного ими предмета обучения.

In [12]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.subj_id, subject.c.subj_name)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id)
stmt = stmt.join(subject, subject.c.subj_id == exam_marks.c.subj_id)


result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

47


[('Вишнякова', 'Анжела', 10, 'Информатика'),
 ('Казакова', 'Мария', 73, 'Физкультура'),
 ('Жук', 'Оксана', 73, 'Физкультура'),
 ('Жуков', 'Пров', 56, 'История'),
 ('Кондрат', 'Синицин', 10, 'Информатика'),
 ('Лазарев', 'Константин', 56, 'История'),
 ('Афанасьева', 'Анжелика', 12, 'Анализ данных'),
 ('Федоров', 'Гостомысл', 12, 'Анализ данных'),
 ('Блохина', 'Ираида', 22, 'Физика'),
 ('Суханов', 'Константин', 22, 'Физика')]

### 4. Напишите запрос на выдачу списка всех студентов. Для студентов, сдававших экзамены, укажите названия соответствующих предметов обучения.

In [13]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.subj_id, subject.c.subj_name)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id, isouter=True)
stmt = stmt.join(subject, subject.c.subj_id == exam_marks.c.subj_id, isouter=True)


result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

100


[('Иванов', 'Иван', 43, 'Математика'),
 ('Петров', 'Петр', None, None),
 ('Сидоров', 'Вадим', 22, 'Физика'),
 ('Кузнецов', 'Борис', 12, 'Анализ данных'),
 ('Зайцева', 'Ольга', 10, 'Информатика'),
 ('Зайцева', 'Ольга', 10, 'Информатика'),
 ('Ордуханян', 'Рафаэль', 10, 'Информатика'),
 ('Котов', 'Павел', 10, 'Информатика'),
 ('Белкин', 'Вадим', 10, 'Информатика'),
 ('Белкин', 'Вадим', 10, 'Информатика')]

### 5. Напишите запрос на выдачу названий всех предметов, по которым студенты получили только хорошие (4 и 5) оценки. В выходных данных должны быть приведены фамилии студентов, названия предметов и оценка.


In [29]:
em1 = exam_marks.alias("em1")
em2 = exam_marks.alias("em2")

subq = ~exists().where(and_(em2.c.mark < 4, em2.c.subj_id == em1.c.subj_id))

stmt = select(subject.c.subj_name, student.c.surname, student.c.name, em1.c.mark)
stmt = stmt.select_from(em1)
stmt = stmt.join(student, em1.c.student_id == student.c.student_id)
stmt = stmt.join(subject, subject.c.subj_id == em1.c.subj_id)
stmt = stmt.where(subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT subject.subj_name, student.surname, student.name, em1.mark 
FROM exam_marks AS em1 JOIN student ON em1.student_id = student.student_id JOIN subject ON subject.subj_id = em1.subj_id 
WHERE NOT (EXISTS (SELECT * 
FROM exam_marks AS em2 
WHERE em2.mark < :mark_1 AND em2.subj_id = em1.subj_id))
1


[('Обработка данных и моделирование в Excel', 'Мельникова', 'Светлана', 4)]

### 6. Напишите запрос, который выполняет вывод списка университетов с рейтингом, превышающим 300, вместе со значением максимального размера стипендии, получаемой студентами в этих университетах.


In [47]:
stmt = select(university.c.univ_name, university.c.rating, func.max(student.c.stipend))
stmt = stmt.select_from(university)
stmt = stmt.join(student, university.c.univ_id == student.c.univ_id, isouter = True)
stmt = stmt.where(university.c.rating > 300)
stmt = stmt.group_by(university.c.univ_id)
stmt = stmt.order_by(university.c.rating)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT university.univ_name, university.rating, max(student.stipend) AS max_1 
FROM university LEFT OUTER JOIN student ON university.univ_id = student.univ_id 
WHERE university.rating > :rating_1 GROUP BY university.univ_id ORDER BY university.rating
10


[('ФГБОУ "Политех"', 304, 200),
 ('БГУ', 326, 250),
 ('ВГМА', 327, 200),
 ('ФинУ', 330, 250),
 ('НГУ', 345, 250),
 ('ТГУ', 368, 250),
 ('МГТУ им. Баумана', 372, 200),
 ('РГУ', 416, 250),
 ('МФТИ', 500, None),
 ('МГУ', 606, 250)]

### 7. Напишите запрос на выдачу списка студентов (в алфавитном порядке фамилий) вместе со значением рейтинга университета, где каждый из них учится, включив в список и тех студентов, место учебы которых в базе данных не указано.

In [51]:
stmt = select(student.c.surname, student.c.name, university.c.univ_name, university.c.rating)
stmt = stmt.select_from(student)
stmt = stmt.join(university, university.c.univ_id == student.c.univ_id, isouter=True)
stmt = stmt.order_by(student.c.surname)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, university.univ_name, university.rating 
FROM student LEFT OUTER JOIN university ON university.univ_id = student.univ_id ORDER BY student.surname
90


[('Агафонов', 'Василий', 'ВГУ', 296),
 ('Аксенов', 'Валерий', 'ВГУ', 296),
 ('Антонов', 'Анатолий', 'МГУ', 606),
 ('Архипов', 'Андрей', 'МГУ', 606),
 ('Афанасьева', 'Анжелика', 'РГУ', 416),
 ('Баранов', 'Александр', 'ФинУ', 330),
 ('Белкин', 'Вадим', 'ВГУ', 296),
 ('Бирюков', 'Герасим', 'ФинУ', 330),
 ('Бирюков', 'Мартин', 'КФУ', None),
 ('Блохина', 'Ираида', 'МГУ', 606)]

### 8. Напишите запрос для получения списка всех студентов вместе с названиями университетов, в которых они учатся. Отдельным запросом получите записи, расширяющие данный список по сравнению с тем, который был бы получен внутренним соединением.

In [52]:
stmt = select(student.c.surname, student.c.name, university.c.univ_name)
stmt = stmt.select_from(student)
stmt = stmt.join(university, university.c.univ_id == student.c.univ_id, isouter=True)
stmt = stmt.order_by(student.c.surname)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, university.univ_name 
FROM student LEFT OUTER JOIN university ON university.univ_id = student.univ_id ORDER BY student.surname
90


[('Агафонов', 'Василий', 'ВГУ'),
 ('Аксенов', 'Валерий', 'ВГУ'),
 ('Антонов', 'Анатолий', 'МГУ'),
 ('Архипов', 'Андрей', 'МГУ'),
 ('Афанасьева', 'Анжелика', 'РГУ'),
 ('Баранов', 'Александр', 'ФинУ'),
 ('Белкин', 'Вадим', 'ВГУ'),
 ('Бирюков', 'Герасим', 'ФинУ'),
 ('Бирюков', 'Мартин', 'КФУ'),
 ('Блохина', 'Ираида', 'МГУ')]

### 9. Напишите запрос для получения списка всех университетов вместе с фамилиями студентов, в них обучающихся. Отдельным запросом получите записи, расширяющие данный список по сравнению с тем, который был бы получен внутренним соединением.

In [74]:
u1 = university.alias("u1")
u2 = university.alias("u2")

subq = ~exists().where(u2.c.univ_id == student.c.univ_id)


stmt = select(student.c.surname, student.c.name, u1.c.univ_name)
stmt = stmt.select_from(u1)
stmt = stmt.join(student, u1.c.univ_id == student.c.univ_id, isouter=True)
stmt = stmt.where(subq)
stmt = stmt.order_by(student.c.surname)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, u1.univ_name 
FROM university AS u1 LEFT OUTER JOIN student ON u1.univ_id = student.univ_id 
WHERE NOT (EXISTS (SELECT * 
FROM university AS u2 
WHERE u2.univ_id = student.univ_id)) ORDER BY student.surname
1


[(None, None, 'МФТИ')]

### 10. Напишите запрос для получения списка всех университетов вместе с фамилиями преподавателей, в них работающих. Отдельным запросом получите записи, расширяющие данный список по сравнению с тем, который был бы получен внутренним соединением.

In [77]:
stmt = select(lecturer.c.surname, lecturer.c.name, university.c.univ_name)
stmt = stmt.select_from(university)
stmt = stmt.join(lecturer, university.c.univ_id == lecturer.c.univ_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT lecturer.surname, lecturer.name, university.univ_name 
FROM university JOIN lecturer ON university.univ_id = lecturer.univ_id
29


[('Сафонова', 'Лукия', 'ФГБОУ "Политех"'),
 ('Зайцева', 'Юлия', 'ВГУ'),
 ('Мамонтова', 'Акулина', 'НГУ'),
 ('Богданова', 'Оксана', 'ВГМА'),
 ('Брагина', 'Нинель', 'УЛГУ'),
 ('Максимова', 'Иванна', 'ВГУ'),
 ('Дмитриева', 'Оксана', 'УЛГУ'),
 ('Ситникова', 'Светлана', 'ВГМА'),
 ('Крюкова', 'Людмила', 'ВГМА'),
 ('Артемьева', 'Майя', 'ВГМА')]

In [84]:
stmt = select(lecturer.c.surname, lecturer.c.name, university.c.univ_name)
stmt = stmt.select_from(university)
stmt = stmt.join(lecturer, university.c.univ_id == lecturer.c.univ_id, isouter = True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT lecturer.surname, lecturer.name, university.univ_name 
FROM university LEFT OUTER JOIN lecturer ON university.univ_id = lecturer.univ_id
32


[('Алексеев', 'Елисей', 'ВГУ'),
 ('Зайцева', 'Юлия', 'ВГУ'),
 ('Зыкова', 'Оксана', 'ВГУ'),
 ('Колесников', 'Борис', 'ВГУ'),
 ('Максимова', 'Иванна', 'ВГУ'),
 ('Николаев', 'Виктор', 'ВГУ'),
 ('Никонов', 'Иван', 'ВГУ'),
 ('Сорокин', 'Андрей', 'ВГУ'),
 ('Мамонтова', 'Акулина', 'НГУ'),
 (None, None, 'МФТИ')]

In [83]:
u1 = university.alias("u1")
u2 = university.alias("u2")

subq = ~exists().where(u2.c.univ_id == lecturer.c.univ_id)


stmt = select(lecturer.c.surname, lecturer.c.name, u1.c.univ_name)
stmt = stmt.select_from(u1)
stmt = stmt.join(lecturer, u1.c.univ_id == lecturer.c.univ_id, isouter=True)
stmt = stmt.where(subq)
stmt = stmt.order_by(lecturer.c.surname)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT lecturer.surname, lecturer.name, u1.univ_name 
FROM university AS u1 LEFT OUTER JOIN lecturer ON u1.univ_id = lecturer.univ_id 
WHERE NOT (EXISTS (SELECT * 
FROM university AS u2 
WHERE u2.univ_id = lecturer.univ_id)) ORDER BY lecturer.surname
3


[(None, None, 'МФТИ'), (None, None, 'БГУ'), (None, None, 'ФинУ')]

### 11. Напишите запрос для получения списка всех преподавателей вместе с университетами, в которых они работают. Есть ли отличие списка от того, который был бы получен внутренним соединением? Подтвердите отдельным запросом ваш вывод.

In [85]:
stmt = select(lecturer.c.surname, lecturer.c.name, university.c.univ_name)
stmt = stmt.select_from(lecturer)
stmt = stmt.join(university, university.c.univ_id == lecturer.c.univ_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT lecturer.surname, lecturer.name, university.univ_name 
FROM lecturer JOIN university ON university.univ_id = lecturer.univ_id
29


[('Сафонова', 'Лукия', 'ФГБОУ "Политех"'),
 ('Зайцева', 'Юлия', 'ВГУ'),
 ('Мамонтова', 'Акулина', 'НГУ'),
 ('Богданова', 'Оксана', 'ВГМА'),
 ('Брагина', 'Нинель', 'УЛГУ'),
 ('Максимова', 'Иванна', 'ВГУ'),
 ('Дмитриева', 'Оксана', 'УЛГУ'),
 ('Ситникова', 'Светлана', 'ВГМА'),
 ('Крюкова', 'Людмила', 'ВГМА'),
 ('Артемьева', 'Майя', 'ВГМА')]

In [86]:
stmt = select(lecturer.c.surname, lecturer.c.name, university.c.univ_name)
stmt = stmt.select_from(lecturer)
stmt = stmt.join(university, university.c.univ_id == lecturer.c.univ_id, isouter = True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT lecturer.surname, lecturer.name, university.univ_name 
FROM lecturer LEFT OUTER JOIN university ON university.univ_id = lecturer.univ_id
29


[('Сафонова', 'Лукия', 'ФГБОУ "Политех"'),
 ('Зайцева', 'Юлия', 'ВГУ'),
 ('Мамонтова', 'Акулина', 'НГУ'),
 ('Богданова', 'Оксана', 'ВГМА'),
 ('Брагина', 'Нинель', 'УЛГУ'),
 ('Максимова', 'Иванна', 'ВГУ'),
 ('Дмитриева', 'Оксана', 'УЛГУ'),
 ('Ситникова', 'Светлана', 'ВГМА'),
 ('Крюкова', 'Людмила', 'ВГМА'),
 ('Артемьева', 'Майя', 'ВГМА')]

#### Разницы нет. Значит нет строк в таблице lecturers, где столбец univ_id имеет значение NULL

### 12. Напишите запрос для получения списка всех студентов и оценок, полученных ими на экзаменах. Отдельным запросом получите записи, расширяющие данный список по сравнению с тем, который был бы получен внутренним соединением.

In [3]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.mark)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, exam_marks.mark 
FROM student JOIN exam_marks ON exam_marks.student_id = student.student_id
47


[('Вишнякова', 'Анжела', 1),
 ('Казакова', 'Мария', 4),
 ('Жук', 'Оксана', 4),
 ('Жуков', 'Пров', 1),
 ('Кондрат', 'Синицин', 3),
 ('Лазарев', 'Константин', 4),
 ('Афанасьева', 'Анжелика', 1),
 ('Федоров', 'Гостомысл', 1),
 ('Блохина', 'Ираида', 5),
 ('Суханов', 'Константин', 1)]

In [4]:
stmt = select(student.c.surname, student.c.name,exam_marks.c.mark)
stmt = stmt.select_from(student)
stmt = stmt.join(exam_marks, exam_marks.c.student_id == student.c.student_id, isouter=True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, exam_marks.mark 
FROM student LEFT OUTER JOIN exam_marks ON exam_marks.student_id = student.student_id
100


[('Иванов', 'Иван', 1),
 ('Петров', 'Петр', None),
 ('Сидоров', 'Вадим', 4),
 ('Кузнецов', 'Борис', 5),
 ('Зайцева', 'Ольга', 2),
 ('Зайцева', 'Ольга', 5),
 ('Ордуханян', 'Рафаэль', 2),
 ('Котов', 'Павел', 4),
 ('Белкин', 'Вадим', None),
 ('Белкин', 'Вадим', 5)]

#### Разница между запросами

In [16]:
s1 = student.alias("s1")
s2 = student.alias("s2")

subq = ~exists().where(s2.c.student_id == exam_marks.c.student_id)


stmt = select(s1.c.surname, s1.c.name, exam_marks.c.mark)
stmt = stmt.select_from(s1)
stmt = stmt.join(exam_marks, s1.c.student_id == exam_marks.c.student_id, isouter=True)
stmt = stmt.where(subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT s1.surname, s1.name, exam_marks.mark 
FROM student AS s1 LEFT OUTER JOIN exam_marks ON s1.student_id = exam_marks.student_id 
WHERE NOT (EXISTS (SELECT * 
FROM student AS s2 
WHERE s2.student_id = exam_marks.student_id))
53


[('Петров', 'Петр', None),
 ('Дуб', 'Елизавета', None),
 ('Кудряшова', 'Вера', None),
 ('Журавлева', 'Вера', None),
 ('Рожкова', 'Пелагея', None),
 ('Соколова', 'Ия', None),
 ('Некрасова', 'Вероника', None),
 ('Казакова', 'Ольга', None),
 ('Шубина', 'Елена', None),
 ('Миронова', 'Анна', None)]

### 13. Напишите запрос для получения списка всех экзаменационных оценок вместе с фамилиями получивших их студентов. Есть ли отличие списка от того, который был бы получен внутренним соединением? Подтвердите отдельным запросом ваш вывод.

In [25]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.mark)
stmt = stmt.select_from(exam_marks)
stmt = stmt.join(student, exam_marks.c.student_id == student.c.student_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, exam_marks.mark 
FROM exam_marks JOIN student ON exam_marks.student_id = student.student_id
47


[('Вишнякова', 'Анжела', 1),
 ('Казакова', 'Мария', 4),
 ('Жук', 'Оксана', 4),
 ('Жуков', 'Пров', 1),
 ('Кондрат', 'Синицин', 3),
 ('Лазарев', 'Константин', 4),
 ('Афанасьева', 'Анжелика', 1),
 ('Федоров', 'Гостомысл', 1),
 ('Блохина', 'Ираида', 5),
 ('Суханов', 'Константин', 1)]

In [26]:
stmt = select(student.c.surname, student.c.name, exam_marks.c.mark)
stmt = stmt.select_from(exam_marks)
stmt = stmt.join(student, exam_marks.c.student_id == student.c.student_id, isouter=True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT student.surname, student.name, exam_marks.mark 
FROM exam_marks LEFT OUTER JOIN student ON exam_marks.student_id = student.student_id
47


[('Вишнякова', 'Анжела', 1),
 ('Казакова', 'Мария', 4),
 ('Жук', 'Оксана', 4),
 ('Жуков', 'Пров', 1),
 ('Кондрат', 'Синицин', 3),
 ('Лазарев', 'Константин', 4),
 ('Афанасьева', 'Анжелика', 1),
 ('Федоров', 'Гостомысл', 1),
 ('Блохина', 'Ираида', 5),
 ('Суханов', 'Константин', 1)]

### 14. Напишите запрос для получения полного списка предметов с соответствующими экзаменационными оценками. Есть ли отличие списка от того, который был бы получен внутренним соединением? Подтвердите отдельным запросом ваш вывод.

In [28]:
stmt = select(subject.c.subj_name, exam_marks.c.mark)
stmt = stmt.select_from(subject)
stmt = stmt.join(exam_marks, exam_marks.c.subj_id == subject.c.subj_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT subject.subj_name, exam_marks.mark 
FROM subject JOIN exam_marks ON exam_marks.subj_id = subject.subj_id
47


[('Информатика', 1),
 ('Физкультура', 4),
 ('Физкультура', 4),
 ('История', 1),
 ('Информатика', 3),
 ('История', 4),
 ('Анализ данных', 1),
 ('Анализ данных', 1),
 ('Физика', 5),
 ('Физика', 1)]

In [29]:
stmt = select(subject.c.subj_name, exam_marks.c.mark)
stmt = stmt.select_from(subject)
stmt = stmt.join(exam_marks, exam_marks.c.subj_id == subject.c.subj_id, isouter=True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT subject.subj_name, exam_marks.mark 
FROM subject LEFT OUTER JOIN exam_marks ON exam_marks.subj_id = subject.subj_id
49


[('Анализ данных', 1),
 ('Анализ данных', 1),
 ('Анализ данных', 1),
 ('Анализ данных', 2),
 ('Анализ данных', 5),
 ('Английский язык', 2),
 ('Английский язык', 4),
 ('Английский язык', 4),
 ('Английский язык', 5),
 ('Информатика', None)]

### 15. Напишите запрос для получения полного списка оценок вместе с названиями предметов, по которым они получены. Есть ли отличие списка от того, который был бы получен внутренним соединением? Подтвердите отдельным запросом ваш вывод.

In [30]:
stmt = select(subject.c.subj_name, exam_marks.c.mark)
stmt = stmt.select_from(exam_marks)
stmt = stmt.join(subject, exam_marks.c.subj_id == subject.c.subj_id)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT subject.subj_name, exam_marks.mark 
FROM exam_marks JOIN subject ON exam_marks.subj_id = subject.subj_id
47


[('Информатика', 1),
 ('Физкультура', 4),
 ('Физкультура', 4),
 ('История', 1),
 ('Информатика', 3),
 ('История', 4),
 ('Анализ данных', 1),
 ('Анализ данных', 1),
 ('Физика', 5),
 ('Физика', 1)]

In [31]:
stmt = select(subject.c.subj_name, exam_marks.c.mark)
stmt = stmt.select_from(exam_marks)
stmt = stmt.join(subject, exam_marks.c.subj_id == subject.c.subj_id, isouter=True)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT subject.subj_name, exam_marks.mark 
FROM exam_marks LEFT OUTER JOIN subject ON exam_marks.subj_id = subject.subj_id
47


[('Информатика', 1),
 ('Физкультура', 4),
 ('Физкультура', 4),
 ('История', 1),
 ('Информатика', 3),
 ('История', 4),
 ('Анализ данных', 1),
 ('Анализ данных', 1),
 ('Физика', 5),
 ('Физика', 1)]